In [1]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import classification_report#这个包是评价报告
from sklearn.model_selection import train_test_split
import copy

In [2]:
df = pd.read_csv("spiral_data/2class.txt",delimiter=" ",names=["x1","x2","y"])
x1 = df["x1"]
x2 = df["x2"]
X_train = np.array(df[["x1","x2"]])
y_train = np.array(df["y"]).reshape([-1,1])
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.1,random_state=4)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1801, 2), (201, 2), (1801, 1), (201, 1))

In [3]:
X_train[0],y_train[0]

(array([ 0.36752, -0.05585]), array([1]))

### 初始化

In [4]:
input_size = 2
hidden_size = 50
output_size = 1
std = 1

# rand函数根据给定维度生成[0,1)之间的数据，shape(input_size, hidden_size)
W1 = std * np.random.randn(input_size, hidden_size)   #(2,10)
b1 = np.zeros((1, hidden_size))                       #(1,10)
W2 = std * np.random.randn(hidden_size,output_size)   #(10,1)
b2 = np.zeros((1, output_size))                       #(1,1)

params = {}
params["W1"] = W1
params["b1"] = b1
params["W2"] = W2
params["b2"] = b2

In [5]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.maximum(0,x)

def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), 1 - sigmoid(z))

# ReLU(x)′={0,ifx<0; 1,ifx>0}
def relu_gradient(x):
    x[x<0] = 0
    x[x>0] = 1
    return x

In [6]:
def forward_propagation(params,X):
    z1 = np.dot(X,params["W1"]) + params["b1"]    # (4*2)                      
    a1 = sigmoid(z1)                 # (4*10)    
    z2 = np.dot(a1,params["W2"]) + params["b2"]     # (4*1)
    a2 = sigmoid(z2)     # (4*1)
    return z1,a1,z2,a2
z1,a1,z2,a2 = forward_propagation(params,X_train)

In [7]:
def computeLoss(y,t): # y->predict value t->true value
#     return -np.sum(t*np.log(y+1e-5) + (1-t)*np.log(1-y+1e-5))/len(y)
    return -np.mean(t*np.log(y+1e-5) + (1-t)*np.log(1-y+1e-5))
computeLoss(a2,y_train)

0.9380800984962432

In [8]:
z1.shape,a1.shape,z2.shape,a2.shape

((1801, 50), (1801, 50), (1801, 1), (1801, 1))

In [11]:
alpha = 0.5
m = X_train.shape[0]
iterations = 50000    

for i in range(iterations):
    z1,a1,z2,a2 = forward_propagation(params,X_train)
    dz2 = a2 - y_train  # (4,1)
    dw2 = (1/m)*np.dot(a1.T,dz2) # (10,1)
    db2 = (1/m)*np.sum(dz2,axis=0,keepdims=True) # (1,1)
    dz1 = np.dot(dz2,params["W2"].T)*sigmoid_gradient(z1) # (4,10)
    dw1 = (1/m)*np.dot(X_train.T,dz1) # (2,10)
    db1 = (1/m)*np.sum(dz1,axis=0,keepdims=True) # (1,10)
    loss = computeLoss(a2,y_train)
    a2[a2 <= 0.5] = 0
    a2[a2 > 0.5] = 1
    acc = np.mean(a2 == y_train)
    if i % 1000 == 0:
        print("iteration %d: loss %f acc %f" % (i, loss,acc))
#         print(a2)
#         print(params)
        
    params["W1"] += -alpha * dw1  # (2,10)
    params["b1"] += -alpha * db1  # (1,10)
    params["W2"] += -alpha * dw2  # (10,1)
    params["b2"] += -alpha * db2  # (1,1)

iteration 0: loss 0.159957 acc 0.950583
iteration 1000: loss 0.157335 acc 0.952804
iteration 2000: loss 0.154683 acc 0.953914
iteration 3000: loss 0.151962 acc 0.955025
iteration 4000: loss 0.149083 acc 0.956691
iteration 5000: loss 0.145818 acc 0.957801
iteration 6000: loss 0.142088 acc 0.959467
iteration 7000: loss 0.138524 acc 0.960577
iteration 8000: loss 0.135175 acc 0.962243
iteration 9000: loss 0.131949 acc 0.964464
iteration 10000: loss 0.128822 acc 0.966130
iteration 11000: loss 0.125766 acc 0.966685
iteration 12000: loss 0.122730 acc 0.968906
iteration 13000: loss 0.119605 acc 0.970017
iteration 14000: loss 0.116248 acc 0.972238
iteration 15000: loss 0.112716 acc 0.972793
iteration 16000: loss 0.109338 acc 0.974459
iteration 17000: loss 0.106219 acc 0.977790
iteration 18000: loss 0.103270 acc 0.977235
iteration 19000: loss 0.100403 acc 0.977790
iteration 20000: loss 0.097565 acc 0.978345
iteration 21000: loss 0.094730 acc 0.978345
iteration 22000: loss 0.091906 acc 0.980011
i

KeyboardInterrupt: 